## Author: Farzan Memarian

In [5]:
# Data generation

import numpy as np
from itertools import permutations
import random

Nexam = 10**5
Ndim = 19
Nperm = 30
x1 = np.random.multivariate_normal(mean= np.ones(Ndim), cov = np.identity(Ndim),size = Nexam)
y1 = np.ones(Nexam)
x2 = np.random.multivariate_normal(mean= -np.ones(Ndim), cov = np.identity(Ndim),size = Nexam)
y2 = -np.ones(Nexam)

X = np.concatenate((x1,x2),axis=0)
y = np.concatenate((y1,y2))


from sklearn import model_selection
X_tr_orig, X_test_orig, y_tr_orig, y_test_orig = model_selection.train_test_split(X,y,test_size=0.5)

# reshaping y
y_tr_orig = y_tr_orig.reshape((len(y_tr_orig),1))
y_test_orig = y_test_orig.reshape((len(y_test_orig),1))

perms = [] # array storing different premutatins of X, Y
for _ in range(Nperm):
    inx = np.random.permutation(Nexam)
    X_perm = X_tr_orig[inx]
    y_perm = y_tr_orig[inx]
    perms.append([X_perm,y_perm])

### a) Batch newton algorithm with the Gauss-Newton approximation

In [ ]:
# training batch newton algo
import math
from numpy.linalg import multi_dot
from numpy import outer, matmul
from numpy.linalg import inv, norm
import time
Nsizes = 5

n_ex_float = np.floor(np.logspace(3.0, 5.0, num=Nsizes))
n_ex = [int(item) for item in n_ex_float]
# n_ex = [400,2000]

theta_store2 = []
time_storage = []
for N in n_ex:
    print "RUNNING FOR {} EXAMPLES".format(N)
    start_time = time.time()
    theta_store1 = []
    thresh = 1.0/N
    perm_counter = 0
    for X_tr,y_tr in perms:
        theta = np.random.uniform(-0.5, 0.5, size=Ndim).reshape((Ndim,1))
        perm_counter += 1
        print "perm counter:", perm_counter 
#         g = np.zeros(Ndim)
        h = np.zeros((Ndim,Ndim))
        keep_iter = True
        counter = 0
        while keep_iter:
            counter += 1
            
#             if counter % 10 == 0:
#                 print "iter:", counter
#                 print "error", norm(d_theta)
#                 print "threshold", thresh
            
            f = 1.71 * np.tanh(0.66 * matmul(X_tr[:N,:], theta))
            df = 1.71 * 0.66 * X_tr[:N,:] / ( np.cosh(0.66 * matmul(X_tr[:N,:], theta)) )**2
            g_mat = 2 * np.diag(f - 1.5*y_tr[:N]) * df
            g = np.sum(g_mat,axis=0).reshape((Ndim,1))
            for i in range(N):
                h += matmul(np.outer(np.transpose(df[i,:]),df[i,:]), np.outer(np.transpose(X_tr[i,:]),X_tr[i,:]))
            h_inv = inv(h)
            d_theta = -matmul(h_inv,g)
            if norm(d_theta) > thresh:
                theta += d_theta
            else:
                keep_iter = False
        theta_store1.append(theta)
    end_time = time.time()
    elapsed_time = end_time - start_time
    time_storage.append(elapsed_time)
    theta_store2.append(theta_store1)

In [8]:
store_mse

[[2.686865920070526, 400], [2.1033582225138647, 2000]]

In [9]:
norm(theta_store2[0][0] - theta_store2[0][10])

2.1180865946899168